In [1]:
import dill
import numpy as np
import scanner_interpretation as sc
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
from corner import corner
import astropy.visualization as vis
rd = sc.Reduced("pkl_jugs/n4/reduced_complete.pkl")
rd_old = sc.Reduced("pkl_jugs/reduced_correctedP_020923.pkl")
rd_old.reduced["BERNOULLI_SIGMA"] = sc.n_sigmas(rd_old.reduced["BERNOULLI_P"])


def get_leftover(redu) -> np.ndarray:

# pval_0_mask = redu.reduced["LAMBDA_RATIO_SIGNIFICANCE"]==0.
    bernP_nan_mask = np.isnan(redu.reduced["BERNOULLI_P"])
    expP_nan_mask = np.isnan(redu.reduced["EXP_CORRECTED_P"])
    bernP_finite_mask = np.isfinite(redu.reduced["BERNOULLI_P"])
    expP_finite_mask = np.isfinite(redu.reduced["EXP_CORRECTED_P"])
    bernP_0_mask = redu.reduced["BERNOULLI_P"] == 0
    expP_0_mask = redu.reduced["EXP_CORRECTED_P"] == 0

    sanebern = ~bernP_nan_mask * bernP_finite_mask * ~bernP_0_mask
    saneexp = ~expP_nan_mask * expP_finite_mask * ~expP_0_mask

    leftover = sanebern * saneexp
    return leftover

l_n4 = get_leftover(rd)
l_n3 = get_leftover(rd_old)

In [3]:
from os import getcwd
getcwd()

'/lustre/fs22/group/hess/user/wybouwt/git/gamma_transients/testing'

In [12]:
cd = rd.getCandidate(21080)
cd = sc.Candidate(cd[cd["BERNOULLI_SIGMA"] > 5])
with open("pkl_jugs/plotter_21080/multiplets.pkl","wb") as f:
    dill.dump([cd.mplet],f)

fig,ax = cd.ToAScatter(0.2)
fig.savefig("figures/ToA_21080.png",facecolor="white")

row is a Table of length 1, not a row. Taking first!
hess1


In [20]:
sixplets = rd.reduced[rd.reduced["Nmax"] == 6]
sixplets.sort("dt")
sixplets

Nmax,OBS_ID,ID,RA,DEC,TIME,ENERGY,dt,da,MEDIAN_RA,MEDIAN_DEC,SkyCoord,MEDIAN_GLAT,MEDIAN_GLON,TEVCAT_SOURCE_NAME,TEVCAT_SOURCE_TYPE,TEVCAT_DISTANCES_DEG,TEVCAT_FLUX_CRAB,DS_INDEX,ANGULAR_MEASURE_DEG,ALT_PNT,PNT_SOURCE,PNT_DISTANCE,BKG_DT_LAMBDA,BKG_PHOTONS,MPLET_DT_LAMBDA,LAMBDA_RATIO_SIGNIFICANCE,BELL_FRACTION,EXP_CORRECTED_P,BERNOULLI_P,BERNOULLI_SIGMA
,,,,,,,,,,,"deg,deg",deg,deg,,,,,,,,,,,,,,,,,
int64,uint32,object,object,object,object,object,float64,float64,float64,float64,SkyCoord,float64,float64,str21,str22,float64,object,int64,float64,float64,str18,float64,float64,int64,float64,float64,float64,float64,float64,float64
6,20276,[6524055323063 6528350289956 6528350289986 6528350290028 6528350290073\n 6528350290143],[165.97131 166.06232 166.14052 166.23145 166.18765 166.12132],[38.28908 38.328835 38.34844 38.336254 38.12073 38.232487],['2004-04-14T20:23:24.467085177' '2004-04-14T20:23:25.401650960'\n '2004-04-14T20:23:25.589681918' '2004-04-14T20:23:25.907663399'\n '2004-04-14T20:23:26.202385718' '2004-04-14T20:23:26.665803248'],[0.83006215 0.84927547 1.0574031 1.196126 1.374775 2.1121292 ],2198718080.0,0.29516464816180843,166.13092041015625,38.3089599609375,"166.13092041015625,38.3089599609375",65.0124155469847,179.59672382925038,Markarian 421,HBL,0.1254141575419944,0.3,0,0.1008041280961167,28.4676796247559,Fornax A east lobe,0.40009903659199114,0.03175152011569988,53,2.274052351662325e-09,4.468866388052106e-10,0.00312551180832088,0.003422948556621926,0.0034170971897715674,2.9274903006695943
6,131748,[8723078580592 8744553415160 8744553416922 8744553418289 8744553419307\n 8744553419351],[329.75275 329.59027 329.67902 329.67795 329.56467 329.63208],[-31.053356 -31.260921 -31.114906 -30.999647 -31.039358 -31.292704],['2017-07-03T02:22:33.226009900' '2017-07-03T02:22:33.274048621'\n '2017-07-03T02:22:34.291750247' '2017-07-03T02:22:35.042723233'\n '2017-07-03T02:22:35.600638920' '2017-07-03T02:22:35.636415297'],[0.2030702 0.36068767 0.21866065 0.22185808 0.50748175 0.2135898 ],2410405376.0,0.29662429310257776,329.655029296875,-31.084131240844727,"329.655029296875,-31.084131240844727",-52.27889545638176,16.32606720985433,PKS 2155-304,HBL,0.8648770810555702,0.15,1,0.1277656386921624,82.94920515427,Fornax A east lobe,0.966951959025126,0.011892003025017512,15,2.0743398625903424e-09,2.233768725545815e-12,0.0035242997109889984,1.517363098343305e-05,1.5173516996647152e-05,4.326154274642353
6,130899,[20242180871582 20242180871667 20242180872392 20242180872398\n 20242180872531 20246475836029],[329.53943 329.67438 329.45215 329.65604 329.6059 329.44778],[-30.205076 -30.20866 -30.220171 -30.253277 -30.309416 -30.31212 ],['2017-06-02T03:57:59.828201348' '2017-06-02T03:57:59.871485764'\n '2017-06-02T03:58:00.249743516' '2017-06-02T03:58:00.253293568'\n '2017-06-02T03:58:00.333277995' '2017-06-02T03:58:02.242071444'],[0.6077716 0.2368164 0.13487929 0.4209663 0.18240058 0.25513366],2413870080.0,0.2490952262719296,329.5726623535156,-30.236724853515625,"329.5726623535156,-30.236724853515625",-52.123174891811416,17.690086319297937,PKS 2155-304,HBL,0.14769063376449015,0.15,1,0.09753051963570815,81.4579985302602,Fornax A east lobe,0.5043378431793415,0.03528920808111067,59,2.071362501356411e-09,1.451686881814851e-09,0.0030668810941278934,0.011331832856901189,0.011267872124889666,2.5342797971349613
6,142636,[15105399986972 15109694947357 15109694949143 15109694950739\n 15109694951231 15109694951722],[83.532295 83.51031 83.45117 83.63036 83.56658 83.52143 ],[21.989128 22.041983 21.839342 22.06081 21.841759 21.83971 ],['2018-10-16T02:53:29.525474126' '2018-10-16T02:53:29.664364392'\n '2018-10-16T02:53:30.602400595' '2018-10-16T02:53:31.508652503'\n '2018-10-16T02:53:31.765909249' '2018-10-16T02:53:32.042157466'],[0.6729215 0.41002157 0.7705771 1.7820694 0.5848025 0.28790423],2516683520.0,0.2848818846991208,83.52686309814453,21.915443420410156,"83.52686309814453,21.915443420410156",-5.920603102131634,184.5885017394697,Crab,PWN,0.14061998272119222,1.0

In [10]:
cd.mplet["dt"] / 1e6

141.695232

In [ ]:
significant_table = rd.reduced[rd.reduced["BERNOULLI_SIGMA"] > 5]
significant_table.sort("BERNOULLI_SIGMA")
significant_table

In [14]:
rd_old.getCandidate(35007)

Nmax,OBS_ID,ID,RA,DEC,TIME,ENERGY,dt,da,MEDIAN_RA,MEDIAN_DEC,SkyCoord,MEDIAN_GLAT,MEDIAN_GLON,TEVCAT_SOURCE_NAME,TEVCAT_SOURCE_TYPE,TEVCAT_DISTANCES_DEG,DS_INDEX,ANGULAR_MEASURE_DEG,BKG_DT_LAMBDA,BKG_PHOTONS,MPLET_DT_LAMBDA,LAMBDA_RATIO_SIGNIFICANCE,PNT_DISTANCE,ALT_PNT,BELL_FRACTION,EXP_CORRECTED_P,BERNOULLI_P,BERNOULLI_SIGMA
,,,,,,,,,,,"deg,deg",deg,deg,,,,,,,,,,,,,,,
int64,uint32,object,object,object,object,object,float64,float64,float64,float64,SkyCoord,float64,float64,str21,str22,float64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
4,35007,[12300786336303 12300786336309 12300786336358 12305081303052],[0.58403856 0.526088 0.45749614 0.4166997 ],[-31.82828 -31.751549 -31.768085 -31.768494],['2006-09-24T22:28:53.679578835' '2006-09-24T22:28:53.709489638'\n '2006-09-24T22:28:53.971949393' '2006-09-24T22:28:54.285595948'],[0.40224195 0.19593626 0.30666494 0.5964279 ],606017024.0,0.177698162902297,0.49179208278656006,-31.76828956604004,"0.49179208278656006,-31.76828956604004",-78.28313920052501,6.65861855121281,NGC 253,Starburst,13.000918161333757,0,0.056336131063859166,0.002343300317735979,4,4.950355255066865e-09,5.345723863570129e-13,1.1460470590280671,81.2128422884387,0.0008089494076557457,1.582010297340254e-05,1.5819978994802142e-05,4.316950259487835


In [ ]:
plt.figure()
plt.yscale("log")
h = plt.hist(rd.reduced[l_n4]["BERNOULLI_SIGMA"],histtype="step",color="m",bins="fd",label="$da = 0.4$",density=True)
h2 = plt.hist(rd_old.reduced[l_n3]["BERNOULLI_SIGMA"],histtype="step",color="c",bins="fd",label="$da = 0.2$",density=True)
plt.legend()
plt.xlabel("sigma")


In [ ]:
for var,label in zip(["PNT_DISTANCE","ALT_PNT"],[r"$d(\mathrm{mplet}, PNT)$ [deg]",r"Pointing altitude [deg]"]):
    pearson, spearman = pearsonr(rd.reduced[leftover]["BERNOULLI_SIGMA"].data,rd.reduced[leftover][var].data)[0],spearmanr(rd.reduced[leftover]["BERNOULLI_SIGMA"].data,rd.reduced[leftover][var].data)[0]
    fig = corner(rd.reduced[leftover].to_pandas(),var_names=[var,"BERNOULLI_SIGMA"],labels=[label,r"$\sigma$"])
    fig.text(.55,.7,f"Pearson correlation: {np.round(pearson,2)}\nSpearman rank correlation: {round(spearman,2)}",fontsize="large")
    fig.set_size_inches(4,4)
    for ax in fig.get_axes():
        ax.spines["right"].set_visible(True)
        ax.spines["top"].set_visible(True)
    plt.savefig("figures/combined/pnt-significance-correlation.png",facecolor="white",dpi=300)